In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data = '/content/dataset_directory/image_data/train/'
val_data = '/content/dataset_directory/image_data/val/'
test_data = '/content/dataset_directory/image_data/test/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
num_classes = 6
img_width, img_height = 224, 224
batch_size = 16
epochs = 20

In [ ]:
import zipfile

zip_path = '/content/drive/MyDrive/mosquito_data.zip'
extract_path = '/content/dataset_directory'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(img_width, img_height, 3),
    include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
base_model.trainable = False

In [ ]:
datagen = ImageDataGenerator(
    samplewise_center=True,
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

datagen_test = ImageDataGenerator(
    samplewise_center=True,
    rescale=1.0 / 255)


In [ ]:
train_generator = datagen.flow_from_directory(
    train_data,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)

Found 1440 images belonging to 6 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    val_data,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)

Found 150 images belonging to 6 classes.


In [ ]:
test_generator = datagen_test.flow_from_directory(
    test_data,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical'
)

Found 90 images belonging to 6 classes.


In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/20
58/90 [==================>...........] - ETA: 5:40 - loss: 1.7938 - accuracy: 0.1940

In [ ]:
model.save('before_model.h5')

In [ ]:
before_model = keras.models.load_model('before_model.h5')

In [ ]:
base_model.trainable = True
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

In [ ]:
model.save('/content/drive/MyDrive/after_model.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
after_model = keras.models.load_model('/content/drive/MyDrive/after_model.h5')

In [ ]:
test_loss, test_accuracy = after_model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
import os

true_labels = []
image_filenames = []

for class_label in os.listdir(test_data):
    class_dir = os.path.join(test_data, class_label)
    if os.path.isdir(class_dir):
        # 'class_label' contains the true label, and 'class_dir' is the directory for that class
        for image_filename in os.listdir(class_dir):
            if image_filename.endswith('.jpeg'):  # Adjust file extension as needed
                true_labels.append(class_label)
                image_filenames.append(os.path.join(class_label, image_filename))


In [ ]:
predicted_labels = after_model.predict(test_generator)

In [ ]:
from sklearn.metrics import classification_report
predicted_labels = after_model.predict(test_generator)

y_pred_labels = [y.argmax() for y in predicted_labels]

y_true = test_generator.classes

target_names = ["Aedes aegypti", "Aedes albopictus", "Anopheles", "Culex", "Culiseta", "Aedes japonicus/Aedes koreicus"]

report = classification_report(y_true, y_pred_labels, target_names=target_names)
print(report)

In [ ]:
class_labels = sorted(set(true_labels))
accuracies = []

for i, label in enumerate(class_labels):
    true_class = np.array(true_labels) == label
    predicted_class_probabilities = predicted_labels[:, i]
    predicted_class_labels = np.argmax(predicted_labels, axis=1)
    predicted_class = predicted_class_labels == i
    accuracy = np.mean(true_class == predicted_class)
    accuracies.append(accuracy)

colors = ['#FFCA3A', '#ADB2D3', '#D16666', '#ACFCD9', '#7D84B2', '#F7C59F']

plt.figure(figsize=(10, 6))
bars = plt.bar(class_labels, accuracies)
for i in range(len(bars)):
    bars[i].set_color(colors[i])
    print(f'{class_labels[i]} : {accuracies[i]}')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.title('Accuracy for Each Class')
plt.xticks(rotation=45)
plt.ylim(0, 1)  # Set the y-axis limit to [0, 1] for accuracy values
plt.show()

In [ ]:
import shutil


source_path = 'after_model.h5'
destination_path = '/content/drive/My Drive/'
shutil.copy(source_path, destination_path)

In [ ]:
from keras.preprocessing import image

def classify_new_image(model, image_path, class_labels):
    # Load and preprocess the new image
    img = image.load_img(image_path, target_size=(244, 244))
    img = image.img_to_array(img)
    img = img / 255.0  # Rescale to match the training data preprocessing
    img = np.expand_dims(img, axis=0)

    # Make predictions
    predictions = model.predict(img)

    # Interpret predictions
    class_indices = np.argmax(predictions, axis=1)
    predicted_class = class_labels[class_indices[0]]

    return predicted_class

In [ ]:
#model = keras.models.load_model('your_model.h5')  # Load your saved model
class_labels = ["Aedes aegypti", "Aedes albopictus", "Anopheles", "Culex", "Culiseta", "Aedes japonicus/Aedes koreicus"]
image_path = '/content/dataset_directory/image_data/test/aegypti/aegypti_240.jpeg'  # Replace with the path to your new image

predicted_class = classify_new_image(after_model, image_path, class_labels)

print(f'The predicted class is: {predicted_class}')